In [ ]:
import json
import os
import numpy as np
import pandas as pd
from research.utils.data_access_utils import RDSAccessUtils
import plotly.express as px
from matplotlib import pyplot as plt

<h1> Get raw biomass computations </h1>

In [ ]:
query = """
    select * from prod.biomass_computations
    where pen_id=37
    and captured_at > '2020-06-15';
"""

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
df = rds_access_utils.extract_from_database(query)

In [ ]:
df.index = list(range(df.shape[0]))
df = df.sort_values('captured_at').copy(deep=True)
df.index = pd.to_datetime(df.captured_at)
dates = df.index.date.astype(str)
df['date'] = dates
df['hour'] = df.index.hour

In [ ]:
mask = (df.captured_at >= '2020-06-13') & (df.captured_at < '2020-06-19') & (df.akpd_score > 0.99)

In [ ]:
mask = (df.captured_at >= '2020-06-10') & (df.captured_at < '2020-06-18') & (df.akpd_score > 0.99)
means = []
hours = np.arange(0, 24, 1)
for hour in hours:
    hour_mask = mask & (df.hour == hour)
    if df[hour_mask].shape[0]:
        means.append(df[hour_mask].estimated_weight_g.mean())
    else:
        means.append(None)
        

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(hours, means)
plt.grid()
plt.show()

In [ ]:
mask = (df.captured_at >= '2020-06-18') & (df.captured_at < '2020-06-19') & (df.akpd_score > 0.99)
means = []
sample_sizes = []
hours = np.arange(0, 24, 1)
for hour in hours:
    hour_mask = mask & (df.hour == hour)
    if df[hour_mask].shape[0]:
        means.append(df[hour_mask].estimated_weight_g.mean())
        sample_sizes.append(df[hour_mask].shape[0])
    else:
        means.append(None)
        sample_sizes.append(None)
        

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(hours, means)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(hours, sample_sizes)
plt.grid()
plt.show()